KNeighborsTimeSeriesClassifier

- requires your data to be in a long format, with two indices where location and date will be the two indices and temperature the only actual column in the dataframe. You can set multi index with : df.set_index(['location', 'date'], inplace=True)


location | date || temperature

London | 2/2/2024 || 15
Belgrade | 2/2/2024. || 21

- if you have n number of locations then for the exact same day, you'll have n number of rows - this is expected as per :https://www.sktime.net/en/v0.29.1/examples/02_classification.html

- the KNeigboursTimeSeriesClassifier wants date duplicates and won't get confused between them 

In [48]:
import pandas as pd 
import numpy as np
from sktime.classification.early_classification._teaser import TEASER
from sktime.classification.interval_based import TimeSeriesForestClassifier

## Concatenating all df

In [49]:
df_london = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\.data\london_weather_2014-2024.csv")
df_london.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,0,"London, England, United Kingdom",2014-01-01,11.6,6.7,9.1,11.6,3.6,7.0,6.7,...,0.5,0,NaN,2014-01-01T08:06:14,2014-01-01T16:02:04,0.00,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,EGWU,03680099999,03768399999,03672..."
1,1,"London, England, United Kingdom",2014-01-02,11.0,6.6,9.1,11.0,3.6,7.2,6.0,...,3.3,2,NaN,2014-01-02T08:06:07,2014-01-02T16:03:08,0.03,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,03672099999,0378109999..."
2,2,"London, England, United Kingdom",2014-01-03,10.6,7.3,9.0,10.6,3.2,6.5,5.1,...,2.8,2,NaN,2014-01-03T08:05:56,2014-01-03T16:04:16,0.07,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,03680099999,03672099999,0378109999..."
3,3,"London, England, United Kingdom",2014-01-04,10.1,4.5,8.0,10.1,2.0,5.1,5.2,...,1.4,1,NaN,2014-01-04T08:05:42,2014-01-04T16:05:25,0.10,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,03680099999,03672099999,0378109999..."
4,4,"London, England, United Kingdom",2014-01-05,12.0,1.8,6.0,12.0,-0.9,3.6,3.9,...,2.6,2,NaN,2014-01-05T08:05:24,2014-01-05T16:06:38,0.14,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,03672099999,0378109999..."


In [65]:
# np.array(df_london['temp'])

In [53]:
df_bujanovac = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\.data\bujanovac_weather_2014-2024.csv")
df_bujanovac.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,0,"Bujanovac, Serbia",2014-01-01,6.6,2.4,4.3,6.6,0.0,3.8,2.1,...,6.6,4,NaN,2014-01-01T07:02:35,2014-01-01T16:10:28,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"LWSK,13493099999,13586099999,BKPR,13389099999,..."
1,1,"Bujanovac, Serbia",2014-01-02,7.6,1.6,3.7,7.6,0.5,3.5,1.0,...,5.5,3,NaN,2014-01-02T07:02:39,2014-01-02T16:11:20,0.03,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"LWSK,13493099999,13586099999,BKPR,13389099999,..."
2,2,"Bujanovac, Serbia",2014-01-03,5.7,1.4,3.3,5.7,1.4,3.2,1.4,...,5.4,3,NaN,2014-01-03T07:02:41,2014-01-03T16:12:15,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"LWSK,13493099999,13586099999,BKPR,13389099999,..."
3,3,"Bujanovac, Serbia",2014-01-04,4.0,0.8,2.5,4.0,-2.2,1.7,1.3,...,7.8,4,NaN,2014-01-04T07:02:41,2014-01-04T16:13:11,0.10,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"LWSK,13493099999,13586099999,BKPR,13389099999,..."
4,4,"Bujanovac, Serbia",2014-01-05,5.3,0.7,3.3,5.3,-2.6,1.2,1.8,...,1.9,1,NaN,2014-01-05T07:02:38,2014-01-05T16:14:08,0.14,"Rain, Overcast",Cloudy skies throughout the day with late afte...,rain,"13493099999,LWSK,13586099999,BKPR,13389099999,..."


In [54]:
df_manila = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\.data\manila_weather_2014-2024.csv")
df_manila.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,0,"Manila, Phillipines",2014-01-01,31.0,23.0,26.6,32.7,23.0,27.4,20.4,...,18.0,8,NaN,2014-01-01T06:21:07,2014-01-01T17:37:49,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"98429099999,98428099999,98426041231,RPLL,98430..."
1,1,"Manila, Phillipines",2014-01-02,31.0,23.6,27.0,33.1,23.6,27.9,20.0,...,19.1,8,NaN,2014-01-02T06:21:29,2014-01-02T17:38:24,0.02,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"98429099999,98428099999,98426041231,RPLL,98430..."
2,2,"Manila, Phillipines",2014-01-03,31.0,22.0,26.7,33.7,22.0,27.9,20.6,...,20.4,8,NaN,2014-01-03T06:21:50,2014-01-03T17:38:58,0.06,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"98429099999,98428099999,98426041231,RPLL,98430..."
3,3,"Manila, Phillipines",2014-01-04,30.0,23.0,26.7,33.6,23.0,28.0,21.8,...,20.2,8,NaN,2014-01-04T06:22:11,2014-01-04T17:39:33,0.09,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"98429099999,98428099999,98426041231,RPLL,98430..."
4,4,"Manila, Phillipines",2014-01-05,30.0,23.0,26.5,32.7,23.0,27.7,21.6,...,19.1,8,NaN,2014-01-05T06:22:30,2014-01-05T17:40:08,0.13,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"98429099999,98428099999,98426041231,RPLL,98430..."


In [55]:
df_brisbane = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\.data\brisbane_weather_2014-2024.csv")
df_brisbane.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,0,"Brisbane, Australia",2014-01-01,29.2,21.9,25.1,29.6,21.9,25.3,18.1,...,32.4,10,NaN,2014-01-01T04:55:58,2014-01-01T18:46:25,0.00,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,"94578099999,94575099999,94595199999,YAMB,94576..."
1,1,"Brisbane, Australia",2014-01-02,29.7,20.9,25.3,31.3,20.9,25.9,18.9,...,29.7,10,NaN,2014-01-02T04:56:39,2014-01-02T18:46:40,0.02,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"94578099999,94575099999,94595199999,YAMB,94576..."
2,2,"Brisbane, Australia",2014-01-03,32.5,22.8,27.5,36.1,22.8,29.6,21.8,...,30.3,10,NaN,2014-01-03T04:57:21,2014-01-03T18:46:54,0.05,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"94578099999,94575099999,94595199999,YAMB,94576..."
3,3,"Brisbane, Australia",2014-01-04,35.8,24.6,29.2,41.3,24.6,31.7,22.0,...,31.8,10,NaN,2014-01-04T04:58:03,2014-01-04T18:47:05,0.09,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"94578099999,94575099999,94595199999,YAMB,94576..."
4,4,"Brisbane, Australia",2014-01-05,31.5,23.8,26.8,35.8,23.8,28.2,21.5,...,29.4,10,NaN,2014-01-05T04:58:46,2014-01-05T18:47:16,0.12,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"94578099999,94575099999,94595199999,YAMB,94576..."


In [56]:
df_all = pd.concat([df_london, df_manila, df_bujanovac, df_brisbane])
df_all.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,0,"London, England, United Kingdom",2014-01-01,11.6,6.7,9.1,11.6,3.6,7.0,6.7,...,0.5,0,NaN,2014-01-01T08:06:14,2014-01-01T16:02:04,0.00,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,EGWU,03680099999,03768399999,03672..."
1,1,"London, England, United Kingdom",2014-01-02,11.0,6.6,9.1,11.0,3.6,7.2,6.0,...,3.3,2,NaN,2014-01-02T08:06:07,2014-01-02T16:03:08,0.03,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,03672099999,0378109999..."
2,2,"London, England, United Kingdom",2014-01-03,10.6,7.3,9.0,10.6,3.2,6.5,5.1,...,2.8,2,NaN,2014-01-03T08:05:56,2014-01-03T16:04:16,0.07,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,03680099999,03672099999,0378109999..."
3,3,"London, England, United Kingdom",2014-01-04,10.1,4.5,8.0,10.1,2.0,5.1,5.2,...,1.4,1,NaN,2014-01-04T08:05:42,2014-01-04T16:05:25,0.10,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"03769099999,03680099999,03672099999,0378109999..."
4,4,"London, England, United Kingdom",2014-01-05,12.0,1.8,6.0,12.0,-0.9,3.6,3.9,...,2.6,2,NaN,2014-01-05T08:05:24,2014-01-05T16:06:38,0.14,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,03672099999,0378109999..."


In [ ]:
df_all.iloc[0, 'dim_0'] = np.array(df_london['temp'])

In [57]:
df_all = df_all[['name', 'datetime', 'temp']]

In [58]:
df_all.sort_values('datetime')

,name,datetime,temp
0,"London, England, United Kingdom",2014-01-01,9.1
0,"Manila, Phillipines",2014-01-01,26.6
0,"Brisbane, Australia",2014-01-01,25.1
0,"Bujanovac, Serbia",2014-01-01,4.3
1,"Brisbane, Australia",2014-01-02,25.3
...,...,...,...
3791,"London, England, United Kingdom",2024-05-19,16.6
3792,"Manila, Phillipines",2024-05-20,31.4
3792,"London, England, United Kingdom",2024-05-20,15.1
3792,"Bujanovac, Serbia",2024-05-20,20.0


## Giving all locations one number

In [59]:
df_all['name'] = np.where(df_all['name']=='London, England, United Kingdom', 1, np.where(df_all['name'] ==' Bujanovac, Serbia', 1, 0))

In [60]:
df_all

,name,datetime,temp
0,1,2014-01-01,9.1
1,1,2014-01-02,9.1
2,1,2014-01-03,9.0
3,1,2014-01-04,8.0
4,1,2014-01-05,6.0
...,...,...,...
3788,0,2024-05-16,18.5
3789,0,2024-05-17,18.4
3790,0,2024-05-18,18.9
3791,0,2024-05-19,16.2


https://github.com/sktime/sktime/blob/main/examples/classification/early_classification.ipynb

In [61]:
# convert datetime column to timestamp not string as it is now 

df_all['datetime'] = pd.to_datetime(df_all['datetime'])

In [62]:
df_all = df_all.set_index('datetime')

In [64]:
df_all

,name,temp
datetime,,
2014-01-01,1,9.1
2014-01-02,1,9.1
2014-01-03,1,9.0
2014-01-04,1,8.0
2014-01-05,1,6.0
...,...,...
2024-05-16,0,18.5
2024-05-17,0,18.4
2024-05-18,0,18.9


## Splitting the data

In [63]:
full_train = df_all[df_all['datetime'] < pd.to_datetime('2023-05-01')]

X_train = full_train.drop('temp',axis=1)

KeyError: 'datetime'

In [ ]:
y_train = full_train[['temp']]

In [ ]:
full_test = df_all[df_all['datetime'] >= pd.to_datetime('2023-05-01')]

In [ ]:
X_test = full_test.drop('temp',axis=1)
y_test = full_test[['temp']]

In [ ]:
clf = TimeSeriesForestClassifier(n_estimators=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
manila = pd.Series(df_manila['temp'])
london = pd.Series(df_london['temp'])
brisbane = pd.Series(df_brisbane['temp'])
bujanovac = pd.Series(df_bujanovac['temp'])

In [ ]:
df_all.loc[0, 'temp'] = manila
df_all.loc[1, 'temp'] = london
df_all.loc[0, 'temp'] = brisbane
df_all.loc[1, 'temp'] = bujanovac

In [ ]:
manila_train = pd.Series(df_manila['temp'])[:-365]
manila_test = pd.Series(df_manila['temp'])[-365:]

london_train = pd.Series(df_london['temp'])
london_test = pd.Series(df_london['temp'])

bujanovac_train = pd.Series(df_bujanovac['temp'])
bujanovac_test = pd.Series(df_bujanovac['temp'])

brisbane_train = pd.Series(df_brisbane['temp'])
brisbane_test = pd.Series(df_brisbane['temp'])

In [ ]:
df_train = pd.DataFrame(columns=['temp'])
df_test = pd.DataFrame(columns=['temp'])

In [ ]:
df_train.loc[0, 'temp'] = manila_train
df_test.loc[0, 'temp'] = manila_test


df_train.loc[0, 'temp'] = london_train
df_test.loc[0, 'temp'] = london_test

In [ ]:
y_train = np.array([1,1,0,0])
y_test = np.array([1,1,0,0])

In [ ]:
df_train_X.loc[0, 'temp'] = manila_train
df_test_X.loc[0, 'temp'] = manila_test


df_train_X.loc[1, 'temp'] = london_train
df_test_X.loc[2, 'temp'] = london_test

df_train_X.loc[1, 'temp'] = bujanovac_train
df_test_X.loc[2, 'temp'] = bujanovac_test

df_train_X.loc[1, 'temp'] = brisbane_train
df_test_X.loc[2, 'temp'] = brisbane_test